<a href="https://colab.research.google.com/github/VKonstantinV/Git_Kon/blob/main/SQL_project.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import sqlalchemy as sa
# устанавливаем параметры
db_config = {
'user': 'praktikum_student', # имя пользователя
'pwd': 'Sdf4$2;d-d30pp', # пароль
'host': 'rc1b-wcoijxj3yxfsf3fs.mdb.yandexcloud.net',
'port': 6432, # порт подключения
'db': 'data-analyst-final-project-db' # название базы данных
}
connection_string = 'postgresql://{user}:{pwd}@{host}:{port}/{db}'.format(**db_config)
# сохраняем коннектор
engine = sa.create_engine(connection_string, connect_args={'sslmode':'require'})
# чтобы выполнить SQL-запрос, пишем функцию с использованием Pandas
def get_sql_data(query:str, engine:sa.engine.base.Engine=engine) -> pd.DataFrame:
  '''Открываем соединение, получаем данные из sql, закрываем соединение'''
  with engine.connect() as con:
    return pd.read_sql(sql=sa.text(query), con = con)

## SQL проект
Наша компания решила быть на волне и купила крупный сервис для чтения книг по подписке. Ваша первая задача как аналитика — проанализировать базу данных. В ней — информация о книгах, издательствах, авторах, а также пользовательские обзоры книг. Эти данные помогут сформулировать ценностное предложение для нового продукта.

Задания
- Посчитайте, сколько книг вышло после 1 января 2000 года;
- Для каждой книги посчитайте количество обзоров и среднюю оценку;
- Определите издательство, которое выпустило наибольшее число книг толще
50 страниц — так вы исключите из анализа брошюры;
- Определите автора с самой высокой средней оценкой книг — учитывайте
только книги с 50 и более оценками;
- Посчитайте среднее количество обзоров от пользователей, которые поставили
больше 48 оценок.

#### Запустили все таблицы.

In [2]:
query = '''SELECT * FROM books LIMIT 5'''
get_sql_data(query)

,book_id,author_id,title,num_pages,publication_date,publisher_id
0,1,546,'Salem's Lot,594,2005-11-01,93
1,2,465,1 000 Places to See Before You Die,992,2003-05-22,336
2,3,407,13 Little Blue Envelopes (Little Blue Envelope...,322,2010-12-21,135
3,4,82,1491: New Revelations of the Americas Before C...,541,2006-10-10,309
4,5,125,1776,386,2006-07-04,268


In [3]:
query = '''SELECT * FROM authors LIMIT 5'''
get_sql_data(query)

,author_id,author
0,1,A.S. Byatt
1,2,Aesop/Laura Harris/Laura Gibbs
2,3,Agatha Christie
3,4,Alan Brennert
4,5,Alan Moore/David Lloyd


In [4]:
query = '''SELECT * FROM publishers LIMIT 5'''
get_sql_data(query)

,publisher_id,publisher
0,1,Ace
1,2,Ace Book
2,3,Ace Books
3,4,Ace Hardcover
4,5,Addison Wesley Publishing Company


In [5]:
query = '''SELECT * FROM ratings LIMIT 5'''
get_sql_data(query)

,rating_id,book_id,username,rating
0,1,1,ryanfranco,4
1,2,1,grantpatricia,2
2,3,1,brandtandrea,5
3,4,2,lorichen,3
4,5,2,mariokeller,2


In [6]:
query = '''SELECT * FROM reviews LIMIT 5'''
get_sql_data(query)

,review_id,book_id,username,text
0,1,1,brandtandrea,Mention society tell send professor analysis. ...
1,2,1,ryanfranco,Foot glass pretty audience hit themselves. Amo...
2,3,2,lorichen,Listen treat keep worry. Miss husband tax but ...
3,4,3,johnsonamanda,Finally month interesting blue could nature cu...
4,5,3,scotttamara,Nation purpose heavy give wait song will. List...


#### Посчитаем, сколько книг вышло после 1 января 2000 года

In [7]:
query = '''
SELECT COUNT(publication_date)
FROM books
WHERE publication_date > '2000-01-01'
'''
get_sql_data(query)

,count
0,819


После 1 января 2000 года вышло 819 книг.

#### Посчитаем количество обзоров и среднюю оценку для каждой книги.

In [8]:
query = '''
SELECT b.book_id, b.title, COUNT(r.review_id) AS review_count, AVG(rt.rating) AS avg_rating
FROM books AS b
LEFT JOIN reviews AS r ON b.book_id = r.book_id
LEFT JOIN ratings AS rt ON b.book_id = rt.book_id
GROUP BY b.book_id
ORDER BY avg_rating DESC, review_count DESC;
'''
get_sql_data(query)

,book_id,title,review_count,avg_rating
0,17,A Dirty Job (Grim Reaper #1),16,5.00
1,553,School's Out—Forever (Maximum Ride #2),12,5.00
2,444,Moneyball: The Art of Winning an Unfair Game,9,5.00
3,347,In the Hand of the Goddess (Song of the Liones...,6,5.00
4,610,Tai-Pan (Asian Saga #2),4,5.00
...,...,...,...,...
995,915,The World Is Flat: A Brief History of the Twen...,12,2.25
996,202,Drowning Ruth,9,2.00
997,371,Junky,4,2.00
998,316,His Excellency: George Washington,4,2.00


Всего оценены 1000 книг, самый высокий рейтинг и самое большое количество обзоров у A Dirty Job (Grim Reaper #1) - 16 обзоров, School's Out—Forever (Maximum Ride #2) - 12 обзоров и Moneyball: The Art of Winning an Unfair Game - 9 обзоров. Так же можем видеть и те книги, у которых рейтинг самый низкий, например Harvesting the Heart - рейтинг 1.5, но и оценок всего 4.

#### Определим издательство, которое выпустило наибольшее число книг толще 50 страниц.



In [9]:
query = '''
SELECT p.publisher_id, p.publisher, COUNT(b.*) AS count
FROM publishers AS p
JOIN books AS b ON p.publisher_id = b.publisher_id
WHERE b.num_pages > 50
GROUP BY p.publisher_id, p.publisher
ORDER BY count DESC
LIMIT 10;
'''
get_sql_data(query)

,publisher_id,publisher,count
0,212,Penguin Books,42
1,309,Vintage,31
2,116,Grand Central Publishing,25
3,217,Penguin Classics,24
4,33,Ballantine Books,19
5,35,Bantam,19
6,45,Berkley,17
7,284,St. Martin's Press,14
8,46,Berkley Books,14
9,83,Delta,13


Нашли издательство, которое выпустило больше всего книг - это Penguin Books. Издательство выпустило 42 книги.

#### Определим автора с самой высокой средней оценкой книг. Будем учитывать только книги с 50 и более оценками.

In [10]:
query = '''
SELECT a.author, a.author_id, COUNT(b.book_id), ROUND(AVG(r.avg_rating), 2) AS author_avg_rating
FROM authors AS a
JOIN books AS b ON a.author_id = b.author_id
JOIN (SELECT book_id, AVG(rating) AS avg_rating, COUNT(*)
  FROM ratings
  GROUP BY book_id
  HAVING COUNT(*) > 50) AS r on b.book_id = r.book_id
GROUP BY a.author_id, a.author
ORDER BY author_avg_rating desc
LIMIT 10;
'''
get_sql_data(query)

,author,author_id,count,author_avg_rating
0,J.K. Rowling/Mary GrandPré,236,4,4.28
1,J.R.R. Tolkien,240,2,4.26
2,Markus Zusak/Cao Xuân Việt Khương,402,1,4.26
3,Louisa May Alcott,376,1,4.19
4,Rick Riordan,498,1,4.08
5,William Golding,621,1,3.90
6,J.D. Salinger,235,1,3.83
7,Paulo Coelho/Alan R. Clarke/Özdemir İnce,469,1,3.79
8,William Shakespeare/Paul Werstine/Barbara A. M...,630,1,3.79
9,Lois Lowry,372,1,3.75


Определили автора с самой высокой оценкой, им оказалась J.K. Rowling/Mary GrandPré с оценкой 4.28.

#### Посчитаем среднее количество обзоров от пользователей, которые поставили больше 48 оценок.

In [15]:
query = '''
SELECT AVG(review_count)
FROM (SELECT r.username, COUNT(rev.review_id) AS review_count
  FROM (SELECT username
    FROM ratings
    GROUP BY username
    HAVING COUNT(*) > 48) AS r
  LEFT JOIN reviews AS rev ON r.username = rev.username
  GROUP BY r.username) AS user_reviews;
'''
get_sql_data(query)

,avg
0,24.0


Посчитали количество пользователей, которые оставляют обзоры на книги и которые оставили не меньше 48 оценок, их число оказалось 24.

#### Вывод

1. Книги, вышедшие после 2000 года
После 1 января 2000 года вышло 819 книг.

2. Количество обзоров и средняя оценка книг
Из 1000 оценённых книг самые популярные книги (по количеству обзоров и высокому рейтингу):  
  - A Dirty Job – 16 обзоров, рейтинг 5.00  
  - School's Out—Forever – 12 обзоров, рейтинг 5.00  
  - Moneyball – 9 обзоров, рейтинг 5.00

Низкорейтинговые книги: например, Harvesting the Heart имеет рейтинг 1.5 и всего 4 оценки.

3. Лидирующее издательство
Наибольшее число книг - 42, выпустило издательство Penguin Books.

4. Автор с самой высокой средней оценкой
Среди авторов книг, имеющими не менее 50 оценок, лидером стала J.K. Rowling / Mary GrandPré со средним рейтингом 4.28.

5. Активные пользователи, которые пишут обзоры*
Среди тех, кто поставил более 48 оценок, только 24 пользователя написали хотя бы один обзор.